In [17]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder

In [18]:
# --- 1. Sinh dữ liệu giả lập ---
n = 1000  # số lượng đơn hàng

data = pd.DataFrame({
    'Order_ID': range(1, n + 1),
    'Delivery_person_ID': np.random.randint(1, 50, n),
    'Distance_km': np.round(np.random.uniform(0.5, 10, n), 2),
    'Time_of_day': np.random.choice(['Morning', 'Afternoon', 'Evening', 'Night'], n),
    'Day_of_week': np.random.choice(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'], n),
    'Weather_condition': np.random.choice(
        ['Clear', 'Rainy', 'Snowy', 'Foggy'], n, p=[0.6, 0.25, 0.1, 0.05]
    ),
    'Traffic_density': np.random.choice(
        ['Low', 'Medium', 'High', 'Jam'], n, p=[0.3, 0.4, 0.2, 0.1]
    ),
    'Order_preparation_time': np.random.randint(5, 30, n)  # thời gian chuẩn bị (phút)
})

In [19]:
# Tính thời gian giao hàng thực tế (Delivery_time) dựa trên các yếu tố + nhiễu Gaussian
base_time = 10  # phút cố định
data['Delivery_time'] = (
    base_time
    + data['Distance_km'] * 2
    + data['Order_preparation_time']
    + data['Weather_condition'].map({'Clear': 0, 'Rainy': 5, 'Snowy': 8, 'Foggy': 3})
    + data['Traffic_density'].map({'Low': 0, 'Medium': 3, 'High': 6, 'Jam': 10})
    + np.random.normal(0, 2, n)  # thêm nhiễu
).round(2)

In [20]:
# Hiện thử 5 dòng đầu
print("5 dòng đầu của dataset giả lập:")
print(data.head(), "\n")

5 dòng đầu của dataset giả lập:
   Order_ID  Delivery_person_ID  Distance_km Time_of_day Day_of_week  \
0         1                  12         8.64       Night         Thu   
1         2                  42         1.38     Evening         Thu   
2         3                  13         9.45       Night         Sun   
3         4                  12         3.70   Afternoon         Fri   
4         5                   4         7.42       Night         Fri   

  Weather_condition Traffic_density  Order_preparation_time  Delivery_time  
0             Rainy          Medium                       8          39.60  
1             Clear             Jam                      26          50.96  
2             Clear          Medium                       8          40.15  
3             Clear          Medium                      29          44.92  
4             Clear            High                      29          61.49   



In [21]:
# --- 2. Tách X, y và train/test split ---
from sklearn.model_selection import train_test_split

# X: đặc trưng, bỏ Order_ID và cột mục tiêu
X = data.drop(columns=['Order_ID', 'Delivery_time'])
y = data['Delivery_time']

for col in X.select_dtypes(include='object').columns:
    X[col] = LabelEncoder().fit_transform(X[col])

# Tách 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Số mẫu train: {len(X_train)}, số mẫu test: {len(X_test)}\n")

Số mẫu train: 800, số mẫu test: 200



In [22]:
# --- 3. Tiền xử lý dữ liệu ---
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Khai báo cột phân loại và cột số
categorical_features = [
    'Time_of_day', 'Day_of_week', 'Weather_condition', 'Traffic_density'
]
numerical_features = [
    'Distance_km', 'Order_preparation_time'
]

# ColumnTransformer: One-Hot cho categorical, passthrough cho numerical
preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_features)
], remainder='passthrough')

In [23]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [24]:
lasso = Lasso(alpha=0.01)  # alpha nhỏ để giữ hiệu năng tốt
lasso.fit(X_train, y_train)


Lasso(alpha=0.01)

In [25]:
dt_params = {'max_depth': [5, 10, 20, None]}
dt = GridSearchCV(DecisionTreeRegressor(random_state=42), dt_params, cv=5)
dt.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=42),
             param_grid={'max_depth': [5, 10, 20, None]})

In [26]:
rf_params = {'n_estimators': [100, 200],
             'max_depth': [10, 20, None]}
rf = GridSearchCV(RandomForestRegressor(random_state=42), rf_params, cv=5)
rf.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
             param_grid={'max_depth': [10, 20, None],
                         'n_estimators': [100, 200]})

In [27]:
xgb_params = {
    'n_estimators': [100, 200],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1]
}
xgb = GridSearchCV(XGBRegressor(random_state=42, objective='reg:squarederror'), xgb_params, cv=5)
xgb.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, feature_weights=None,
                                    gamma=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bin=None,
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None, ...),
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 6, 10], 'n_estimators': [100, 200],
                         'subsample': [0.8, 1]})

In [28]:
models = {
    'Linear Regression': lr,
    'Lasso': lasso,
    'Decision Tree': dt.best_estimator_,
    'Random Forest': rf.best_estimator_,
    'XGBoost': xgb.best_estimator_,
}

for name, model in models.items():
    y_pred = model.predict(X_test)
    score = r2_score(y_test, y_pred)
    print(f"{name} R² Score: {score}")


Linear Regression R² Score: 0.9024206074215225
Lasso R² Score: 0.9024019544155324
Decision Tree R² Score: 0.8470789255181278
Random Forest R² Score: 0.9185519910361097
XGBoost R² Score: 0.9579862688920795
